#### Optimal Opponent Experiments
Author: Yemi Kelani

##### Google Drive Setup (Skip if running locally)

> To run this notebook, follow these steps:
> 1. Download the latest version of the [repository](https://github.com/yemi-kelani/artificial-intelligence/tree/master).
> 2. Upload the repsitory files to your Google Drive account under the path `Projects/artificial-intelligence`.
> 3. Open this file (`self-train.ipynb`) from your Google Drive and run the experiments.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
ROOT_FOLDER = "drive/MyDrive/Projects/artificial-intelligence/models/ReinforcementLearning/"
PROJECT_PATH = f"{ROOT_FOLDER}/DeepQ_TicTacToe_v2"
NOTEBOOK_LOCATION = f"{PROJECT_PATH}/experiments"

In [ ]:
!cp {PROJECT_PATH}/DeepQAgent.py .
!cp {PROJECT_PATH}/TicTacToeGame.py .
!cp {ROOT_FOLDER}/Utils.py .

from DeepQAgent import DeepQAgent
from  TicTacToeGame import TicTacToeGame, OPPONENT_LEVEL
from Utils import (
    train_agent,
    test_agent,
    set_seed
)
MODEL_PATH = "drive/MyDrive/Projects/artificial-intelligence/trained_models/ReinforcementLearning/TicTacToeV2"

##### Local Setup (Skip if running remotely)
> 1. Run the following cells

In [ ]:
from models.ReinforcementLearning.DeepQ_TicTacToe_v2.DeepQAgent import DeepQAgent
from models.ReinforcementLearning.DeepQ_TicTacToe_v2.TicTacToeGame import TicTacToeGame, OPPONENT_LEVEL
from models.ReinforcementLearning.Utils import (
    train_agent,
    test_agent,
    set_seed
)
MODEL_PATH = "../../../../trained_models/ReinforcementLearning/TicTacToeV2"

##### Experiments

In [ ]:
import os
import torch

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(DEVICE)

SEED = 100
set_seed(SEED)

# DeepQ parameters
BATCH_SIZE     = 128
NUM_EPISODES   = 10000 if torch.cuda.is_available() else 100
STATE_SPACE    = 9
ACTION_SPACE   = 9
HIDDEN_SIZE    = 128
EPSILON        = 1.0
GAMMA          = 0.99
LEARNING_RATE  = 0.001
DROPOUT        = 0.25
TRAIN_START = 1500
NEGATIVE_SLOPE = 0.01
COMPUTER_LEVEL = OPPONENT_LEVEL.NAIVE

# save path
MODEL_NAME = ""

BASELINE = "TicTacToev2-Baseline-Untrained"
NAIVE_2K = "TicTacToev2-NAIVE-2K"
NAIVE_4K = "TicTacToev2-NAIVE-4K"
NAIVE_6K = "TicTacToev2-NAIVE-6K"
NAIVE_8K = "TicTacToev2-NAIVE-8K"
NAIVE_10K = "TicTacToev2-NAIVE-10K"
AGENT_1K = "TicTacToev2-AGENT-1K"
AGENT_2K = "TicTacToev2-AGENT-2K"
AGENT_3K = "TicTacToev2-AGENT-3K"
AGENT_4K = "TicTacToev2-AGENT-4K"
OPTIMAL_1K = "TicTacToev2-OPTIMAL-1K"
OPTIMAL_2K = "TicTacToev2-OPTIMAL-2K"
OPTIMAL_4K = "TicTacToev2-OPTIMAL-4K"
OPTIMAL_6K = "TicTacToev2-OPTIMAL-6K"
OPTIMAL_8K = "TicTacToev2-OPTIMAL-8K"
SELF_10K = "TicTacToev2-SELF-10K"


def get_full_model_path(agent_name: str = None):
  if agent_name is None:
    return os.path.join(MODEL_PATH, MODEL_NAME + ".pt")
  return os.path.join(MODEL_PATH, agent_name + ".pt")

def supply_model(load_if_exists: bool = True, agent_name: str = None):

  agent = DeepQAgent(
      device         = DEVICE,
      epsilon        = EPSILON,
      gamma          = GAMMA,
      state_space    = STATE_SPACE,
      action_space   = ACTION_SPACE,
      hidden_size    = HIDDEN_SIZE,
      dropout        = DROPOUT,
      train_start    = TRAIN_START,
      batch_size     = BATCH_SIZE,
      negative_slope = NEGATIVE_SLOPE
  )

  full_model_path = get_full_model_path(agent_name)
  if load_if_exists and os.path.exists(full_model_path):
    print("Loading Model Parameters...")
    agent.load_model(filepath=full_model_path)

  optimizer = torch.optim.Adam(agent.parameters(), lr=LEARNING_RATE)
  criterion = torch.nn.SmoothL1Loss() # Huber Loss

  return agent, optimizer, criterion

def compare_to_naive(agent_name: str, num_episodes: int = 25000):
  _agent, _, _ = supply_model(load_if_exists=True, agent_name=agent_name)
  _environment = TicTacToeGame(DEVICE, None, OPPONENT_LEVEL.NAIVE, start_as_X=False)
  test_agent(_agent, _environment, num_episodes)

def compare_to_optimal(agent_name: str, num_episodes: int = 100):
  _agent, _, _ = supply_model(load_if_exists=True, agent_name=agent_name)
  _environment = TicTacToeGame(DEVICE, None, OPPONENT_LEVEL.OPTIMAL, start_as_X=False)
  test_agent(_agent, _environment, num_episodes)

def compare_to_model(agent_name: str, model_name: str, num_episodes: int = 10000):
  _agent, _, _ = supply_model(load_if_exists=True, agent_name=agent_name)
  _enemy, _, _ = supply_model(load_if_exists=True, agent_name=model_name)
  _environment = TicTacToeGame(DEVICE, _enemy, OPPONENT_LEVEL.AGENT, start_as_X=False)
  test_agent(_agent, _environment, num_episodes)


In [ ]:
agent, optimizer, criterion = supply_model(load_if_exists=True, agent_name=OPTIMAL_4K)
environment = TicTacToeGame(DEVICE, agent, OPPONENT_LEVEL.AGENT)
reward_history = train_agent(
    agent,
    environment,
    NUM_EPISODES,
    optimizer,
    criterion,
    DEVICE,
    MODEL_PATH,
    model_name = SELF_10K
)

In [ ]:
compare_to_naive(SELF_10K)

In [ ]:
compare_to_model(SELF_10K, NAIVE_2K)

In [ ]:
compare_to_optimal(SELF_10K)

In [ ]:
compare_to_model(SELF_10K, OPTIMAL_2K)

In [ ]:
compare_to_model(SELF_10K, OPTIMAL_4K)

In [ ]:
compare_to_model(SELF_10K, SELF_10K)